<a href="https://colab.research.google.com/github/sargegas/sargegas/blob/master/Pruebaperretesygatos1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Nueva sección

In [8]:
import tensorflow as tf
import tensorflow.keras as kr
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K

K.clear_session()

data_entrenamiento="/content/drive/My Drive/Datos/Entrenamiento"
data_validacion="/content/drive/My Drive/Datos/Validacion"

## Parametros.

epocas=20
altura, longitud= 100, 100
batch_size=32
pasos=1000
pasos_validacion=200
filtrosConv1=32
filtrosConv2=64
tamano_filtro1=(3,3)
tamano_filtro2=(2,2)
tamano_pool=(2,2)
clases=2
lr=0.0005


##Preprocesamiento de imagenes.

entrenamiento_datagen= ImageDataGenerator(
    rescale=1./255,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True
)

validacion_datagen= ImageDataGenerator(
    rescale=1./255
)

imagen_entrenamiento= entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode= 'categorical'
)

imagen_validacion= validacion_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode= 'categorical'    
)


## Red convolucional sobre CNN

cnn=Sequential()

cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding='same', input_shape=(altura, longitud, 3), activation='relu'))

cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Flatten()) ## sirve para aplanar la informacion.
cnn.add(Dense(256, activation='relu')) ## capa con 256 neuronas.
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax')) ##capa de salida, con tantas salidas como clases.

cnn.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=lr), metrics=['accuracy']) ##Le decimos a la IA que mientras entrena su fucnion de aprendizaje es categorical, despues el lernig rate.

cnn.fit_generator(imagen_entrenamiento, steps_per_epoch=pasos, epochs=epocas, validation_data=imagen_validacion, validation_steps=pasos_validacion) ## aqui decimos que entrenamos con imagen entrenamiendo, con sus x pasos cada epoca.

##Guardamos las imagenes en un archivo para no tener que entrenar siempre a la IA desde 0

dir='/content/drive/My Drive/Datos/modelo'

if not os.path.exists(dir): ## Si no existe la carpeta modelo, la creamos.
  os.mkdir(dir)

cnn.save('/content/drive/My Drive/Datos/modelo/modelo.h5')
cnn.save_weights('/content/drive/My Drive/Datos/modelopesos.h5')






Found 212 images belonging to 2 classes.
Found 212 images belonging to 2 classes.
Epoch 1/20
1000/1000 [==============================] - 784s 784ms/step - loss: 0.2456 - acc: 0.8905 - val_loss: 0.0066 - val_acc: 1.0000
Epoch 2/20
1000/1000 [==============================] - 733s 733ms/step - loss: 0.0259 - acc: 0.9934 - val_loss: 0.0034 - val_acc: 1.0000
Epoch 3/20
1000/1000 [==============================] - 725s 725ms/step - loss: 0.0133 - acc: 0.9965 - val_loss: 0.0022 - val_acc: 1.0000
Epoch 4/20
1000/1000 [==============================] - 721s 721ms/step - loss: 0.0111 - acc: 0.9968 - val_loss: 9.3493e-05 - val_acc: 1.0000
Epoch 5/20
1000/1000 [==============================] - 719s 719ms/step - loss: 0.0078 - acc: 0.9977 - val_loss: 5.1971e-06 - val_acc: 1.0000
Epoch 6/20
1000/1000 [==============================] - 717s 717ms/step - loss: 0.0077 - acc: 0.9975 - val_loss: 1.1721e-04 - val_acc: 1.0000
Epoch 7/20
1000/1000 [==============================] - 715s 715ms/step - loss

In [21]:
import numpy as np

from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model

altura, longitud= 100, 100
modelo = '/content/drive/My Drive/Datos/modelo/modelo.h5'
pesos = '/content/drive/My Drive/Datos/modelopesos.h5'

cnn=load_model (modelo)  ##fallo en las dos CNN. no se por que.
cnn.load_weights (pesos)


def predict(file):
  x=load_img(file, target_size=(longitud, altura))
  x=img_to_array(x)
  x=np.expand_dims(x, axis=0)
  arreglo=cnn.predict(x)   ## prediccion de la imagen que llamamos antes. de 2 dimensiones.[1, 0] por ejemplo
  resultado=arreglo[0]
  respuesta=np.argmax(resultado)
  if respuesta==0:
    print('perro')
  elif respuesta==1:
    print('gato')
  return respuesta

predict('f1b7c7599aae10ca398284263ec72c72.jpg')
  



ValueError: ignored